<a href="https://colab.research.google.com/github/keremmust/credit-card-fraud-detection/blob/main/Credit_Card_Fraud_Detection_Experimental_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
## Credit Card Fraud Detection – Experimental Pipeline

##This notebook contains the complete experimental pipeline used in the study. It covers data loading, preprocessing, model training, evaluation, and comparative analysis across multiple machine learning and deep learning models.##


In [23]:
### 1. Data Loading and Train-Test Split

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'creditcardfraud' dataset.
Path to dataset files: /kaggle/input/creditcardfraud


In [3]:
import pandas as pd

df = pd.read_csv(os.path.join(path, "creditcard.csv"))

df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
from sklearn.model_selection import train_test_split

X = df.drop("Class", axis=1)
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train class distribution:")
print(y_train.value_counts())

print("\nTest class distribution:")
print(y_test.value_counts())


Train class distribution:
Class
0    227451
1       394
Name: count, dtype: int64

Test class distribution:
Class
0    56864
1       98
Name: count, dtype: int64


In [24]:
### 2. Baseline Logistic Regression Model


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Scaling
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression
lr = LogisticRegression(
    max_iter=1000,
    random_state=42
)

lr.fit(X_train_scaled, y_train)

# Predictions
y_pred = lr.predict(X_test_scaled)
y_proba = lr.predict_proba(X_test_scaled)[:, 1]

# Metrics
print(classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_proba))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.63      0.72        98

    accuracy                           1.00     56962
   macro avg       0.91      0.82      0.86     56962
weighted avg       1.00      1.00      1.00     56962

AUC-ROC: 0.9605494455801453


In [25]:
### 3. Handling Class Imbalance with SMOTE


In [7]:
from imblearn.over_sampling import SMOTE

# SMOTE
smote = SMOTE(random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(
    X_train_scaled, y_train
)

print("Before SMOTE:", y_train.value_counts())
print("After SMOTE:", y_train_smote.value_counts())


Before SMOTE: Class
0    227451
1       394
Name: count, dtype: int64
After SMOTE: Class
0    227451
1    227451
Name: count, dtype: int64


In [33]:
### 4. Logistic Regression Classification with SMOTE

In [34]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Apply SMOTE on scaled training data
smote = SMOTE(random_state=42)
X_train_lr_smote, y_train_lr_smote = smote.fit_resample(
    X_train_scaled, y_train
)

print("After SMOTE (LR):")
print(y_train_lr_smote.value_counts())

# Logistic Regression model
lr_smote = LogisticRegression(
    max_iter=1000,
    random_state=42
)

# Train
lr_smote.fit(X_train_lr_smote, y_train_lr_smote)

# Evaluate on original test set
y_pred_lr_smote = lr_smote.predict(X_test_scaled)
y_proba_lr_smote = lr_smote.predict_proba(X_test_scaled)[:, 1]

print("\nLogistic Regression + SMOTE Classification Report:")
print(classification_report(y_test, y_pred_lr_smote))

print("AUC-ROC:", roc_auc_score(y_test, y_proba_lr_smote))


After SMOTE (LR):
Class
0    227451
1    227451
Name: count, dtype: int64

Logistic Regression + SMOTE Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.97     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.97      0.99     56962

AUC-ROC: 0.9708434302252134


In [26]:
###5. Random Forest - Baseline (NO SMOTE)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Define a lightweight and stable Random Forest model
rf = RandomForestClassifier(
    n_estimators=20,
    max_depth=10,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1
)

# Train the model on the original (imbalanced) training data
rf.fit(X_train, y_train)

# Predictions on the untouched test set
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

# Evaluation
print("Random Forest (Baseline) Classification Report:")
print(classification_report(y_test, y_pred_rf))

print("AUC-ROC:", roc_auc_score(y_test, y_proba_rf))


Random Forest (Baseline) Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.85      0.76      0.80        98

    accuracy                           1.00     56962
   macro avg       0.93      0.88      0.90     56962
weighted avg       1.00      1.00      1.00     56962

AUC-ROC: 0.9665019760718018


In [28]:
### 5. Random Forest Classification with SMOTE


In [14]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Apply SMOTE only on training data
smote = SMOTE(random_state=42)
X_train_rf_smote, y_train_rf_smote = smote.fit_resample(X_train, y_train)

print("After SMOTE (RF):")
print(y_train_rf_smote.value_counts())

# Define Random Forest (same lightweight)
rf_smote = RandomForestClassifier(
    n_estimators=20,
    max_depth=10,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1
)

# Train
rf_smote.fit(X_train_rf_smote, y_train_rf_smote)

# Evaluate on untouched test set
y_pred_rf_smote = rf_smote.predict(X_test)
y_proba_rf_smote = rf_smote.predict_proba(X_test)[:, 1]

print("\nRandom Forest + SMOTE Classification Report:")
print(classification_report(y_test, y_pred_rf_smote))

print("AUC-ROC:", roc_auc_score(y_test, y_proba_rf_smote))


After SMOTE (RF):
Class
0    227451
1    227451
Name: count, dtype: int64

Random Forest + SMOTE Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.47      0.88      0.61        98

    accuracy                           1.00     56962
   macro avg       0.73      0.94      0.80     56962
weighted avg       1.00      1.00      1.00     56962

AUC-ROC: 0.9782059665453124


In [15]:
!pip install xgboost


In [29]:
### 6. Baseline XGBoost Classification Model


In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

xgb = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)
y_proba_xgb = xgb.predict_proba(X_test)[:, 1]

print("XGBoost (Baseline) Classification Report:")
print(classification_report(y_test, y_pred_xgb))

print("AUC-ROC:", roc_auc_score(y_test, y_proba_xgb))


XGBoost (Baseline) Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.89      0.84      0.86        98

    accuracy                           1.00     56962
   macro avg       0.95      0.92      0.93     56962
weighted avg       1.00      1.00      1.00     56962

AUC-ROC: 0.9753973497812181


In [30]:
### 7. XGBoost Classification with SMOTE


In [17]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Apply SMOTE on training data
smote = SMOTE(random_state=42)
X_train_xgb_smote, y_train_xgb_smote = smote.fit_resample(X_train, y_train)

print("After SMOTE (XGBoost):")
print(y_train_xgb_smote.value_counts())

# XGBoost model
xgb_smote = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

# Train
xgb_smote.fit(X_train_xgb_smote, y_train_xgb_smote)

# Evaluate
y_pred_xgb_smote = xgb_smote.predict(X_test)
y_proba_xgb_smote = xgb_smote.predict_proba(X_test)[:, 1]

print("\nXGBoost + SMOTE Classification Report:")
print(classification_report(y_test, y_pred_xgb_smote))

print("AUC-ROC:", roc_auc_score(y_test, y_proba_xgb_smote))


After SMOTE (XGBoost):
Class
0    227451
1    227451
Name: count, dtype: int64

XGBoost + SMOTE Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.52      0.87      0.65        98

    accuracy                           1.00     56962
   macro avg       0.76      0.93      0.83     56962
weighted avg       1.00      1.00      1.00     56962

AUC-ROC: 0.9775299892044607


In [31]:
### 8. Baseline MLP Classification Model

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Scale features (MLP requires scaling)
scaler = StandardScaler()
X_train_mlp = scaler.fit_transform(X_train)
X_test_mlp = scaler.transform(X_test)

mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=20,
    random_state=42
)

mlp.fit(X_train_mlp, y_train)

y_pred_mlp = mlp.predict(X_test_mlp)
y_proba_mlp = mlp.predict_proba(X_test_mlp)[:, 1]

print("MLP (Baseline) Classification Report:")
print(classification_report(y_test, y_pred_mlp))

print("AUC-ROC:", roc_auc_score(y_test, y_proba_mlp))


MLP (Baseline) Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.75      0.83      0.79        98

    accuracy                           1.00     56962
   macro avg       0.87      0.91      0.89     56962
weighted avg       1.00      1.00      1.00     56962

AUC-ROC: 0.972281160635329


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [32]:
### 9. MLP Classification with SMOTE


In [19]:
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Scale features (required for MLP)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE on scaled training data
smote = SMOTE(random_state=42)
X_train_mlp_smote, y_train_mlp_smote = smote.fit_resample(
    X_train_scaled, y_train
)

print("After SMOTE (MLP):")
print(y_train_mlp_smote.value_counts())

# MLP model
mlp_smote = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=30,
    random_state=42
)

mlp_smote.fit(X_train_mlp_smote, y_train_mlp_smote)

# Evaluate
y_pred_mlp_smote = mlp_smote.predict(X_test_scaled)
y_proba_mlp_smote = mlp_smote.predict_proba(X_test_scaled)[:, 1]

print("\nMLP + SMOTE Classification Report:")
print(classification_report(y_test, y_pred_mlp_smote))

print("AUC-ROC:", roc_auc_score(y_test, y_proba_mlp_smote))


After SMOTE (MLP):
Class
0    227451
1    227451
Name: count, dtype: int64

MLP + SMOTE Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.70      0.84      0.76        98

    accuracy                           1.00     56962
   macro avg       0.85      0.92      0.88     56962
weighted avg       1.00      1.00      1.00     56962

AUC-ROC: 0.9681637462244325


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(
